In [ ]:
import os
import sys
import numpy as np
import glob
import pickle
import matplotlib.pyplot as plt
import src.analysis as spc
import MDAnalysis as mda

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
name_sim = "XRD/"
path_base = f"/data/sperez/Projects/string_sims/models/raw_pdb/"
path_processed = f"/data/sperez/Projects/string_sims/data/processed/{name_sim}"
path_raw = f"/data/sperez/Projects/string_sims/data/raw"
path_report = f"/data/sperez/Projects/string_sims/reports/figures/{name_sim}"
os.chdir(path_base)
os.getcwd()

# Load Universes

In [ ]:
universes = {
    "5VKH": {"marker": "o", "string0_bead": 0},
    "3FB5": {"marker": "X", "string0_bead": 5},
    "5VK6": {"marker": "*", "string0_bead": 10},
    "5VKE": {"marker": "s", "string0_bead": 17},
}

In [ ]:
sim_names = ["C2I_lb_v1"]

In [ ]:
for u in universes.keys():
    universes[u]["u"] = mda.Universe(f"{u}.pdb")

## Calculate Stuff

## SF

In [ ]:
for name in universes.keys():
    u = universes[name]["u"]
    sel1 = u.select_atoms("resid 77 and name CA and segid PROA")
    sel2 = u.select_atoms("resid 77 and name CA and segid PROB")
    distance = spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 77 and name CA and segid PROC")
    sel2 = u.select_atoms("resid 77 and name CA and segid PROD")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    universes[name]["SF"] = 0.5 * distance

## IG

In [ ]:
for name in universes.keys():
    u = universes[name]["u"]
    sel1 = u.select_atoms("resid 112 and name CA and segid PROA")
    sel2 = u.select_atoms("resid 112 and name CA and segid PROB")
    distance = spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 112 and name CA and segid PROC")
    sel2 = u.select_atoms("resid 112 and name CA and segid PROD")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    universes[name]["IG"] = 0.5 * distance

## W67-L81

In [ ]:
for name in universes.keys():
    u = universes[name]["u"]
    sel1 = u.select_atoms("resid 67 and name CD1 and segid PROA")
    sel2 = u.select_atoms("resid 81 and name CG and segid PROA")
    distance = spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 67 and name CD1 and segid PROB")
    sel2 = u.select_atoms("resid 81 and name CG and segid PROB")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 67 and name CD1 and segid PROC")
    sel2 = u.select_atoms("resid 81 and name CG and segid PROC")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 67 and name CD1 and segid PROD")
    sel2 = u.select_atoms("resid 81 and name CG and segid PROD")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    universes[name]["W67-L81"] = 0.25 * distance

## I100-F103 

In [ ]:
for name in universes.keys():
    u = universes[name]["u"]
    sel1 = u.select_atoms("resid 100 and name CD and segid PROD")
    sel2 = u.select_atoms("resid 103 and name CZ and segid PROA")
    distance = spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 100 and name CD and segid PROB")
    sel2 = u.select_atoms("resid 103 and name CZ and segid PROD")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 100 and name CD and segid PROC")
    sel2 = u.select_atoms("resid 103 and name CZ and segid PROB")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    sel1 = u.select_atoms("resid 100 and name CD and segid PROA")
    sel2 = u.select_atoms("resid 103 and name CZ and segid PROC")
    distance += spc.distance_atom_groups(u, sel1, sel2)
    universes[name]["I100-F103"] = 0.25 * distance

## Path Values

In [ ]:
for sim_name in sim_names:
    string0 = np.loadtxt(f"{path_raw}/{sim_name}/strings/string0.txt")
    file = glob.glob(f"{path_processed}/../{sim_name}/path_lam*")[0]
    path = np.load(file)
    lam = float(file.split("_")[-1][3:][:-4])
    for name in universes.keys():
        coord = string0[universes[name]["string0_bead"], :]
        path_cvs = spc.cvs_to_path(coord, path=path, lam=lam)
        universes[name][f"s_path_{sim_name}"] = path_cvs[0]
        universes[name][f"z_path_{sim_name}"] = path_cvs[1]

## Save data

In [ ]:
with open(f"{path_processed}XRD.pickle", "wb") as handle:
    pickle.dump(universes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(f"{path_processed}XRD.pickle", "rb") as handle:
#    universes = pickle.load(handle)